# Import Dependencies

In [1]:
import numpy
import pandas
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# Use Pandas to convert CSV to SQLITE

In [2]:
df = pandas.read_csv('../data/pothole_datasd.csv')
df = df[['service_request_id', 'lat', 'lng', 'status']]
connection = sqlite3.connect('potholes.sqlite')
df.to_sql('potholes', con=connection, if_exists='replace')

# SQLAlchemy Prep: Create custom Python Class
Note: Tried using `Base = automap_base()` method but reflection was not working.

Source: [https://github.com/davidjaimes/sqlalchemy-challenge/blob/master/climate.ipynb](https://github.com/davidjaimes/sqlalchemy-challenge/blob/master/climate.ipynb)

In [3]:
Base = declarative_base()

class Potholes(Base):
    __tablename__ = 'potholes'
    service_request_id = Column(Integer, primary_key=True)
    lat =  Column(Float)
    lng =  Column(Float)
    status =  Column(String)

# Create database connection and Begin Queries

In [4]:
engine = create_engine('sqlite:///potholes.sqlite')
Base.metadata.create_all(engine)
session = Session(bind=engine)

### Example 1: Flatten SQL query.

In [5]:
# Query all latitudes and print first element.
latitudes = session.query(Potholes.lat).all()
print(latitudes[0])

# Problem: Value is inside a SET.
# Solution: Use NumPy to turn to array and then flatten.
latitudes = numpy.array(latitudes).flatten()
print(latitudes[0])

(32.831881,)
32.831881


### Example 2: Filter SQL query.

In [6]:
new_requests = session.query(Potholes).filter(Potholes.status == 'In Process')
new_requests.count()

921

### Example 3: Group By